## PROBLEM 1 - OBJECT CLASSIFICATION

In [1]:
import json
import wandb
import os
import sys
import glob
import argparse

from keras import __version__
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, RMSprop
from keras.activations import softmax

import wandb
from wandb.keras import WandbCallback

Using TensorFlow backend.


In [2]:
run = wandb.init()
config = run.config
config.img_width = 299
config.img_height = 299
config.epochs = 30
config.batch_size = 16
config.n_train_samples = 10

W&B Run: https://app.wandb.ai/univai-ss2019/uncategorized/runs/aasejf94
Call `%%wandb` in the cell containing your training loop to display live results.


In [3]:
anno_classes = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'NoF']
train_data_dir = "train"
# Data Augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.1,
        rotation_range=10.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,validation_split=0.2)

# this is the augmentation configuration we will use for validation:
# only rescaling
# val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir, 
        subset = 'training',
        target_size = (config.img_width, config.img_height),
        batch_size = config.batch_size,
        shuffle = True,
        classes = anno_classes,
        class_mode = 'categorical')

Found 3025 images belonging to 8 classes.


In [13]:
valid_generator = train_datagen.flow_from_directory(
        train_data_dir, 
        subset = 'validation',
        target_size = (config.img_width, config.img_height),
        batch_size = config.batch_size,
        shuffle = True,
        classes = anno_classes,
        class_mode = 'categorical')

Found 752 images belonging to 8 classes.


In [15]:
# setup model
conv_base = InceptionV3(include_top=False, weights='imagenet', input_shape=None)


model = Sequential()
model.add(conv_base)
# model.add(Dropout(0.3))
model.add(GlobalAveragePooling2D())
model.add(Dense(100,activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))
# model.add(Flatten(input_shape=conv_base.output_shape[1:]))
# model.add(Dense(256, activation='relu'))
# model.add(Dense(8, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, None, None, 2048)  21802784  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               204900    
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 808       
Total params: 22,008,492
Trainable params: 21,974,060
Non-trainable params: 34,432
_________________________________________________________________


In [17]:
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 192


In [18]:
conv_base.trainable = False

In [19]:
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


In [20]:
# transfer learning
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['acc'])

In [21]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [22]:
nb_train_samples = get_nb_files(train_data_dir)

In [23]:
nb_train_samples

3777

In [24]:
nb_val_samples = get_nb_files(val_dir)

NameError: name 'val_dir' is not defined

In [25]:
model.fit_generator(
    train_generator,
    epochs=config.epochs,
    steps_per_epoch=3025 // config.batch_size,
    validation_data=valid_generator,
    validation_steps=752 // config.batch_size,
    callbacks=[WandbCallback()])

wandb: Wandb version 0.8.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Instructions for updating:
Use tf.cast instead.
Epoch 1/30


ResourceExhaustedError: OOM when allocating tensor with shape[16,64,147,147] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node inception_v3/conv2d_3/convolution}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics/acc/Mean}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


## Problem 2 : Object Detection

In [ ]:
conv_base = InceptionV3(include_top=False, weights='imagenet', input_shape=None)


model = Sequential()
model.add(conv_base)
# model.add(Dropout(0.3))
model.add(GlobalAveragePooling2D())

model.add(Dense(100,activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(8,activation='softmax'))

In [ ]:
import glob
import numpy as np
folders = glob.glob('train/*')
from sklearn.model_selection import train_test_split

anno_classes = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'NoF']
class_idx = list(range(0, 8))

clas_dict = dict(zip(anno_classes, class_idx))


records = []
for folder in folders:
    files = glob.glob(folder+"/*.jpg")
    labels = [e.split('/')[1] for e in files]
    train, valid = train_test_split(range(len(files)), test_size=0.2, random_state=1983)
    mask = np.zeros(len(files))
    for j in train:
        mask[j] = 1
    for i, label in enumerate(labels):
        d = dict(label=clas_dict[label], file=files[i], train=mask[i])
        records.append(d)
import pandas as pd

from keras_preprocessing.image import ImageDataGenerator


import json
bb_json = {}
anno_classes = ['ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT', 'NoF']
anno_classes1 = ['alb', 'bet', 'dol', 'lag', 'other', 'shark', 'yft', 'NoF']
class_idx = list(range(0, 8))

clas_dict = dict(zip(anno_classes, class_idx))

for c in anno_classes1:
    j = json.load(open('bbox/{}_labels.json'.format(c), 'r'))
    for l in j:
        if 'annotations' in l.keys() and len(l['annotations'])>0:
            bb_json[l['filename'].split('/')[-1]] = sorted(
                l['annotations'], key=lambda x: x['height']*x['width'])[-1]
#print(bb_json.keys())
count=0
tot=0
keys = bb_json.keys()
records2 = []
for r in records:
    tot +=1
    name = r['file'].split('/')[-1]
    if not name in keys:
        count += 1
        #print(r['file'])
        r['bbox'] = None
    else:
        bbox = bb_json[name]
        r['x'] = bbox['x']
        r['y'] = bbox['y']
        r['width'] = bbox['width']
        r['height'] = bbox['height']
        records2.append(r)
print("nobbox", count, tot)
print("rec5", records2[:5])
df = pd.DataFrame.from_records(records2)
dftrain = df[df.train==1][['file', 'label', 'x', 'y', 'width', 'height']]
dfvalid = df[df.train==0][['file', 'label', 'x', 'y', 'width', 'height']]

dftrain.to_csv("tv_train.csv", index=False, header=True)
dfvalid.to_csv("tv_valid.csv", index=False, header=True)

In [11]:
from keras.models import Model
from keras.layers import Input, Flatten

In [22]:
input = Input(shape=(299,299,3,))

In [23]:
conv_base1 = InceptionV3(include_top=False, weights='imagenet', 
                         pooling='avg', input_shape=None)(input)

In [24]:
backbone = Dense(100)(conv_base1)

In [25]:
y1 = Dense(8, activation='softmax', name='class_output')(backbone)
y2 = Dense(4, activation='relu', name='bbox_output')(backbone)

In [26]:
model = Model(inputs=input, outputs=[y1, y2])

In [27]:
model.summary() #Keras - NHWC format / PyTorch - NCHW

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
inception_v3 (Model)            (None, 2048)         21802784    input_4[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 100)          204900      inception_v3[1][0]               
__________________________________________________________________________________________________
class_output (Dense)            (None, 8)            808         dense_2[0][0]                    
__________________________________________________________________________________________________
bbox_outpu

In [6]:
import pandas as pd
x= pd.read_csv('tv_train.csv')

In [7]:
x.head()

,file,label,x,y,width,height
0,train/SHARK/img_00478.jpg,5,497.0,364.0,70.0,123.0
1,train/SHARK/img_06913.jpg,5,501.0,368.0,65.0,111.0
2,train/SHARK/img_06410.jpg,5,372.0,575.0,630.0,169.0
3,train/SHARK/img_05873.jpg,5,588.0,343.0,33.0,117.0
4,train/SHARK/img_05399.jpg,5,538.0,364.0,78.0,129.0


In [8]:
x['file'] = x['file'].str.replace('train/','')

In [28]:
# Data Augmentation
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

# this is the augmentation configuration we will use for validation:
# only rescaling
# val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(x,
        train_data_dir,
        x_col = 'file' , y_col = ['label','x','y','width','height']  ,                                           
        subset = 'training',
        target_size = (config.img_width, config.img_height),
        batch_size = config.batch_size,
        shuffle = True,
        classes = anno_classes,
        class_mode = 'other')

Found 2408 images.


In [59]:
# valid_generator = train_datagen.flow_from_dataframe(x,
#         train_data_dir,
#         x_col = 'file' , y_col = ['label','x','y','width','height']  ,                                           
#         subset = 'validation',
#         target_size = (config.img_width, config.img_height),
#         batch_size = config.batch_size,
#         shuffle = True,
#         classes = anno_classes,
#         class_mode = 'other')

Found 601 images.


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:194: UserWarning: `classes` will be ignored given the class_mode="other"
  .format(self.class_mode))


In [ ]:
total_loss = 100* classification_loss + regression_log

y_true - [class , x, y, h , w]
y_pred - 


In [10]:
# import keras.backend as K

# def fish_loss():
#     def custom(y_true, y_pred):
#         true_label, predicted_label = y_true[0], y_pred[0]
#         true_bbox, predicted_bbox = y_true[1:], y_pred[1:]
#         return - (K.mean_squared_error(true_bbox, predicted_bbox) + 
#                   K.categorical_crossentropy(true_label, predicted_label))
#     return custom
# # Finally, you can use it as follows in Keras compile.

# # build model 


# # get the loss function
# model_fish = fish_loss(y_true, y_pred)

# # compile model
# model.compile(loss=model_fish,)

NameError: name 'y_true' is not defined

In [53]:
# transfer learning
model.compile(loss={'class_output': 'sparse_categorical_crossentropy', 'bbox_output': 'mean_squared_error'},
              loss_weights = {'class_output': 10, 'bbox_output': 1},
              optimizer=RMSprop(lr=1e-4))

In [34]:
g = iter(train_generator)

In [37]:
next(g)

(array([[[[0.7254902 , 0.76470596, 0.73333335],
          [0.7372549 , 0.77647066, 0.74509805],
          [0.7411765 , 0.7803922 , 0.7490196 ],
          ...,
          [0.26666668, 0.33333334, 0.3647059 ],
          [0.25882354, 0.3254902 , 0.35686275],
          [0.25882354, 0.3254902 , 0.35686275]],
 
         [[0.7176471 , 0.7568628 , 0.7254902 ],
          [0.7294118 , 0.7686275 , 0.7372549 ],
          [0.7411765 , 0.7803922 , 0.7490196 ],
          ...,
          [0.26666668, 0.33333334, 0.3647059 ],
          [0.26666668, 0.33333334, 0.3647059 ],
          [0.26666668, 0.33333334, 0.3647059 ]],
 
         [[0.7176471 , 0.7568628 , 0.7254902 ],
          [0.7294118 , 0.7686275 , 0.7372549 ],
          [0.7411765 , 0.7803922 , 0.7490196 ],
          ...,
          [0.2784314 , 0.33333334, 0.36862746],
          [0.27058825, 0.3254902 , 0.36078432],
          [0.27058825, 0.3254902 , 0.36078432]],
 
         ...,
 
         [[0.5372549 , 0.58431375, 0.5686275 ],
          [0.53725

In [41]:
Out[37][0].shape

(16, 299, 299, 3)

In [47]:
Out[37][1][:,0]

array([7., 0., 0., 7., 6., 0., 0., 0., 6., 6., 0., 0., 1., 0., 0., 0.])

In [48]:
Out[37][1][:,1:]

array([[462.48      ,  60.63      , 290.46      , 193.17      ],
       [395.        ,  65.        , 200.        , 119.        ],
       [607.96631486, 351.48052578, 129.43032875, 146.05440767],
       [194.73921023, 100.93190774,  91.43243407, 166.24078922],
       [361.6449626 , 383.41063165, 224.35381939, 137.29114321],
       [641.25009573, 478.844719  , 103.80549852, 170.7767879 ],
       [213.06443603, 371.38899719, 112.0062055 ,  92.63671132],
       [819.21      , 276.36      ,  64.86      ,  80.37      ],
       [357.        ,  58.        , 487.        , 196.        ],
       [ 66.        , 318.        , 212.        , 235.        ],
       [184.        , 307.        , 299.        , 121.        ],
       [374.04177574, 397.79045992, 146.05440767,  53.43453939],
       [818.        , 269.        , 198.        , 292.        ],
       [388.        , 264.        , 243.        ,  74.        ],
       [535.8       , 143.82      , 373.65      , 150.87      ],
       [461.91190719, 472

In [31]:
model.fit_generator(
    train_generator,
    epochs=config.epochs,
    steps_per_epoch=2405 // config.batch_size,
    callbacks=[WandbCallback()])

wandb: Wandb version 0.8.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Instructions for updating:
Use tf.cast instead.
Epoch 1/30


ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 2 array(s), but instead got the following list of 1 arrays: [array([[  0.        , 351.        , 439.        , 177.        ,
         67.        ],
       [  0.        , 757.        , 680.        , 424.        ,
         70.        ],
       [  3.        , 528...

In [49]:
model.fit_generator(
    {'input_1': train_generator[0]},
    {'class_output': train_generator[1][:,0], 'bbox_output': train_generator[1][:,1:]},
    epochs=config.epochs,
    steps_per_epoch=2405 // config.batch_size,
    callbacks=[WandbCallback()])

TypeError: tuple indices must be integers or slices, not tuple

In [50]:
g = iter(train_generator)
g1 = next(g)

In [55]:
model.fit(
    {'input_4': g1[0]},
    {'class_output': g1[1][:,0], 'bbox_output': g1[1][:,1:]},
    epochs=config.epochs,
    steps_per_epoch=2405 // config.batch_size,
    callbacks=[WandbCallback()])

Epoch 1/30
150/150 [==============================] - 125s 833ms/step - loss: 103673.6774 - class_output_loss: 3.9405 - bbox_output_loss: 103634.2727


wandb: Wandb version 0.8.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 2/30
150/150 [==============================] - 99s 662ms/step - loss: 62680.0119 - class_output_loss: 3.3234 - bbox_output_loss: 62646.7783
Epoch 3/30
150/150 [==============================] - 99s 662ms/step - loss: 28697.5404 - class_output_loss: 2.8610 - bbox_output_loss: 28668.9302
Epoch 4/30
150/150 [==============================] - 99s 662ms/step - loss: 2311.2270 - class_output_loss: 2.0492 - bbox_output_loss: 2290.7346
Epoch 5/30
150/150 [==============================] - 99s 662ms/step - loss: 204.1265 - class_output_loss: 1.1015 - bbox_output_loss: 193.1113
Epoch 6/30
150/150 [==============================] - 99s 662ms/step - loss: 44.5119 - class_output_loss: 1.0122 - bbox_output_loss: 34.3901
Epoch 7/30
150/150 [==============================] - 99s 663ms/step - loss: 33.2876 - class_output_loss: 1.0102 - bbox_output_loss: 23.1852
Epoch 8/30
150/150 [==============================] - 99s 662ms/step - loss: 27.8825 - class_output_loss: 1.0094 - bbox_output_loss: 17.

In [ ]:
# Train with all batches.
# Go for multiple epochs
# conv_base.trainable = False